In [10]:
from List_to_Pytorch_Dataset import dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
import CNN1D
import Smaller_input_CNN
from statistics import mean

In [19]:
print(f"Dataset size: {len(dataset)}")
print(f"Data sample size: {len(dataset[0][0][0])}")

Dataset size: 2797
Data sample size: 1000


In [12]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda:0 device


In [13]:
labels = { 0  : "STD",
           1  : "WAL",
           2  : "JOG",
           3  : "JUM",
           4  : "STU",
           5  : "STN",
           6  : "SCH",
           7  : "SIT",
           8  : "CHU",
           9  : "CSI",
           10 : "CSO",
           11 : "FOL",
           12 : "FKL",
           13 : "BSC",
           14 : "SDL"}

classification_map = {}

for key, value in labels.items():
    classification_map[value] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def valid_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)

            for key, value in labels.items():
                for i in range(len(y)):
                    if y[i] == key:
                        if pred[i].argmax() == key:
                            classification_map[value]["TP"] += 1
                        else:
                            classification_map[value]["FN"] += 1
                    elif pred[i].argmax() == key:
                        classification_map[value]["FP"] += 1
                    else:
                        classification_map[value]["TN"] += 1
                

            
            
            #model loss and accuracy
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    #model stats
    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n\n")
    return 100*correct

In [16]:
def main():
    model = CNN1D.CNN().to(device)
    model.train()

    learning_rate = 0.001
    epochs = 50

    # Initialize the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #creating datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

    #Dataloaders for the datasets
    train_dataloader = DataLoader(train_set, batch_size = 64, shuffle = True)
    test_dataloader = DataLoader(test_set, batch_size = 64, shuffle = True)
    
    model.train()
    for _ in range(epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer)
    
    model.eval()

    model_accuracy = valid_loop(test_dataloader, model, loss_fn)

    return model_accuracy


In [17]:
accuracies = []
for _ in range(2):
    accuracy = main()
    accuracies.append(accuracy)

print(f"Average accuracy: {mean(accuracies)}")


loss: 2.899162  [    0/ 2237]
loss: 2.722261  [  640/ 2237]
loss: 2.720768  [ 1280/ 2237]
loss: 2.467215  [ 1920/ 2237]
loss: 2.294560  [    0/ 2237]
loss: 2.271122  [  640/ 2237]
loss: 2.232650  [ 1280/ 2237]
loss: 1.812074  [ 1920/ 2237]
loss: 2.009011  [    0/ 2237]
loss: 1.670400  [  640/ 2237]
loss: 1.702777  [ 1280/ 2237]
loss: 1.596867  [ 1920/ 2237]
loss: 1.468806  [    0/ 2237]
loss: 1.510196  [  640/ 2237]
loss: 1.468573  [ 1280/ 2237]
loss: 1.233488  [ 1920/ 2237]
loss: 1.339675  [    0/ 2237]
loss: 1.181476  [  640/ 2237]
loss: 1.223030  [ 1280/ 2237]
loss: 1.178825  [ 1920/ 2237]
loss: 0.966490  [    0/ 2237]
loss: 1.136310  [  640/ 2237]
loss: 1.023296  [ 1280/ 2237]
loss: 1.066477  [ 1920/ 2237]
loss: 0.984348  [    0/ 2237]
loss: 1.106387  [  640/ 2237]
loss: 0.953201  [ 1280/ 2237]
loss: 0.983134  [ 1920/ 2237]
loss: 0.689992  [    0/ 2237]
loss: 0.875749  [  640/ 2237]
loss: 1.014603  [ 1280/ 2237]
loss: 0.868099  [ 1920/ 2237]
loss: 0.768228  [    0/ 2237]
loss: 0.75

In [18]:
stats_map = {}
for key, value in labels.items():
    stats_map[value] = {
        "Sensitivity" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FN"]),
        "Specificity" : float(classification_map[value]["TN"]) / float(classification_map[value]["TN"] + classification_map[value]["FP"]),
        "Recall" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FN"]),
        "Precision" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FP"]),
        "Accuracy" : float(classification_map[value]["TP"] + classification_map[value]["TN"]) / float(classification_map[value]["TP"] + classification_map[value]["TN"] + classification_map[value]["FP"] + classification_map[value]["FN"])
    }
    stats_map[value]["F-score"] = 2.0 / float(1.0 / float(stats_map[value]["Precision"]) + 1.0 / float(stats_map[value]["Recall"]))
print(stats_map)

{'STD': {'Sensitivity': 0.96045197740113, 'Specificity': 0.9843690430804423, 'Recall': 0.96045197740113, 'Precision': 0.8056872037914692, 'Accuracy': 0.9828571428571429, 'F-score': 0.8762886597938144}, 'WAL': {'Sensitivity': 0.7595628415300546, 'Specificity': 0.9766908674054261, 'Recall': 0.7595628415300546, 'Precision': 0.695, 'Accuracy': 0.9625, 'F-score': 0.7258485639686684}, 'JOG': {'Sensitivity': 0.782608695652174, 'Specificity': 0.9912079510703364, 'Recall': 0.782608695652174, 'Precision': 0.8622754491017964, 'Accuracy': 0.9775, 'F-score': 0.8205128205128205}, 'JUM': {'Sensitivity': 0.8932584269662921, 'Specificity': 0.9893211289092296, 'Recall': 0.8932584269662921, 'Precision': 0.8502673796791443, 'Accuracy': 0.9832142857142857, 'F-score': 0.8712328767123287}, 'STU': {'Sensitivity': 0.6818181818181818, 'Specificity': 0.9880860876249039, 'Recall': 0.6818181818181818, 'Precision': 0.8132530120481928, 'Accuracy': 0.9664285714285714, 'F-score': 0.7417582417582418}, 'STN': {'Sensitiv